In [1]:
from sx_multi import DataSource
from sx_multi import Analysis
from sx_multi import Accumulator
from sx_multi import LocalExecutor

from sx_multi import FuncXExecutor

import matplotlib.pyplot as plt



from servicex import ServiceXDataset
from sx_multi import FuncAdlDataset
from coffea import hist, processor


In [2]:
dids = ['mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00']
datasets = [
    ServiceXDataset(did, backend_type='xaod')
    for did in dids
]

In [3]:
leptons_per_event_query = FuncAdlDataset() \
        .Select(lambda e: e.Electrons("Electrons")) \
        .Select(lambda eles: eles.Where(lambda e: e.pt()/1000.0 > 30.0)) \
        .Select(lambda eles: eles.Where(lambda e: abs(e.eta()) < 2.5)) \
        .Where(lambda eles: len(eles) == 2) \
        .Select(lambda ls: (ls.Select(lambda e: e.pt()/1000.0), ls.Select(lambda e: e.eta()), ls.Select(lambda e: e.phi()), ls.Select(lambda e: e.m()/1000.0), ls.Select(lambda e: e.charge()))) \
        .AsROOTTTree('data.root', 'mytree', ('electrons_pt', 'electrons_eta', 'electrons_phi', 'electrons_mass', 'electrons_charge'))

In [4]:
datasource = DataSource(query=leptons_per_event_query, metadata={}, datasets=datasets)

In [5]:
class Z_EEAnalysis(Analysis):
    def __init__(self):
        self.accumulator = Accumulator({
            "mass": hist.Hist(
                "Events",
                hist.Cat("dataset", "Dataset"),
                hist.Bin("mass", "$Z_{ee}$ [GeV]", 60, 60, 120),
            ),
        })

    @staticmethod
    def process(output, events):
        import awkward1 as ak

        dataset = events.metadata['dataset']
        electrons = events.electrons

        # Form the invar mass, plot.
        cut = (ak.num(electrons) == 2)
        diele = electrons[cut][:, 0] + electrons[cut][:, 1]

        output["sumw"][dataset] += len(events)
        output["mass"].fill(
            dataset=dataset,
            mass=diele.mass,
        )

        return output



In [6]:
analysis = Z_EEAnalysis()
executor = FuncXExecutor(endpoint_id='d0a328a1-320d-4221-9d53-142189f77e71')
# executor = LocalExecutor()

Batch result:  {}


In [7]:
%matplotlib widget
async def plot_stream(accumulator_stream):
  async for coffea_info in accumulator_stream:
    # Need to ask coffea folks how to anomate this!
    hist.plot1d(coffea_info['mass'])
    plt.show()
  return coffea_info

await plot_stream(executor.execute(analysis, datasource))

Batch result:  {'ab093e20-ee58-4c1a-a7ba-4dd56cea785c': {'pending': False, 'status': 'success', 'result': {'mass': <Hist (dataset,mass) instance at 0x12ff52850>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 50000.0})}, 'completion_t': '1606340668.3752534'}, '72b1b9f3-6427-486b-8fef-ecddcf14e97c': {'pending': False, 'status': 'success', 'result': {'mass': <Hist (dataset,mass) instance at 0x12fbc9a10>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 150000.0})}, 'completion_t': '1606340670.8643506'}, '9470633c-d314-4aea-b11e-865a310894af': {'pending': False, 'status': 'success', 'result': {'mass': <Hist (dataset,mass) instance at 0x12fef6e50>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 150000.0})}, 'completion_t': '1606340671.0628464'}, '99dfece8-1237-49e9-b08b-b2ef60e3aa71': {'pending': False, 'status': 'success', 'result': {'mass': <Hist (dataset,mass) instance at 0x12ff2d690>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 150000.0}

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'mass': <Hist (dataset,mass) instance at 0x12fbc9a10>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 150000.0})}
{'mass': <Hist (dataset,mass) instance at 0x12fef6e50>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 150000.0})}
{'mass': <Hist (dataset,mass) instance at 0x12ff2d690>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 150000.0})}
{'mass': <Hist (dataset,mass) instance at 0x12fefe550>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 125000.0})}
{'mass': <Hist (dataset,mass) instance at 0x12fef69d0>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 149800.0})}
{'mass': <Hist (dataset,mass) instance at 0x12ff56a10>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 150000.0})}
{'mass': <Hist (dataset,mass) instance at 0x12ff56fd0>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 150000.0})}
{'mass': <Hist (dataset,mass) instance at 0x12ff56ad0>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15

{'mass': <Hist (dataset,mass) instance at 0x12f325d50>,
 'sumw': defaultdict_accumulator(float, {'mc15x': 1993800.0})}